### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_multiple_reports
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20220921_scenario_projections.ipynb")

2022/09/19 11:18:18 INFO|============================================================


### Fit an initial scenario through February 2022

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20220921_sep_gov_briefing/20220921_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/input/vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2024-01-01',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/09/19 11:18:20 INFO|-------------------------|{} Retrieving vaccinations data
2022/09/19 11:18:23 INFO|-------------------------|{} Constructing vaccination projections
2022/09/19 11:18:25 INFO|-------------------------|{} Retrieving hospitalizations data
2022/09/19 11:18:27 INFO|----------------------|{} Prepping Model
2022/09/19 11:18:27 INFO|-----------------------|{} Prepping Model
2022/09/19 11:19:53 INFO|----------------------|{} Model prepped for fitting in 85.810708625 seconds.
2022/09/19 11:19:53 INFO|----------------------|{} Will fit 25 times
2022/09/19 11:21:38 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/25 completed in 105.47238958399998 seconds: {0: {'co': 0.8306366874249484}, 14: {'co': 0.6981959499362199}, 28: {'co': 0.5695644890782656}, 42: {'co': 0.0027700663958021587}, 56: {'co': 0.681471761077721}, 70: {'co': 0.8912752515818526}}
2022/09/19 11:21:38 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/09/19 11

2022/09/19 11:27:20 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/25 completed in 16.516439750000018 seconds: {504: {'co': 0.7818331304673305}, 518: {'co': 0.7754467512044924}, 532: {'co': 0.7520676571972096}, 546: {'co': 0.7034199466923622}, 560: {'co': 0.7375736326007304}, 574: {'co': 0.7317269098323875}}
2022/09/19 11:27:20 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/09/19 11:27:38 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/25 completed in 15.366519957999913 seconds: {532: {'co': 0.7512544747619323}, 546: {'co': 0.705550286968481}, 560: {'co': 0.7337532535187415}, 574: {'co': 0.7437555398822334}, 588: {'co': 0.7812810757641397}, 602: {'co': 0.7595697974466253}}
2022/09/19 11:27:38 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/09/19 11:27:57 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/25 completed in 17.202804499999957 seconds: 

### Create and run scenarios from Feb 2022 to present

In [4]:
multiprocess = 6

scenario_params = json.load(open("covid_model/analysis/20220921_sep_gov_briefing/20220921_scenario_params.json"))

model_args = {
    'base_spec_id': model.spec_id, # use the spec id that was output from the model fit
    'end_date': '2024-01-01',
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# compute vacc effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
#for hosp_frac in [0.7, 0.8, 0.9]:
for beta_mult in [0.95, 1.0, 1.05]:
    for booster_mult in [0, 0.5, 1]:
        for vir_mult in [1, 2.38]:
            hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*0.8),
                   "2022-03-15": (0.34 + 0.66*0.8), "2022-03-30": 0.8}
            beta_param_adjust = [{"param": "betta",
                                  "attrs": {"variant": "ba45"},
                                  "mults": {"2020-01-01": beta_mult},
                                  "desc": "sensitivity analysis for beta"}]
            booster_adjust = [{"param": "immunity",
                               "attrs": {"vacc": "booster3"},
                               "mults": {"2022-08-27": booster_mult},
                               "desc": "booster uptake multiplier"}]
            vir_adjust = [{"param": "hosp",
                           "attrs": {"variant": "vx"},
                           "mults": {"2022-01-01": vir_mult},
                           "desc": "Variant X virulence multiplier"}]
            lt5_vacc_adjust = [{"param": "immunity",
                                "attrs": {'age': '0-19', 'vacc': 'shot1'},
                                "mults": {"2020-01-01": 1, "2022-06-20": 0.99 + 0.01*vacc_eff_lt5,
                                          "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                          "2022-07-10": 0.97 + 0.03*vacc_eff_lt5,
                                          "2022-07-20": 0.96 + 0.04*vacc_eff_lt5,
                                          "2022-07-30": 0.955 + 0.045*vacc_eff_lt5,
                                          "2022-08-10": 0.95 + 0.05*vacc_eff_lt5},
                                "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}]
            scenario_model_args.append({'params_defs': scenario_params + vir_adjust + beta_param_adjust
                                        + booster_adjust + lt5_vacc_adjust,
                                        'hosp_reporting_frac': hrf,
                                        'tags': {'beta_mult': beta_mult,
                                                 'booster_mult': booster_mult,
                                                 'vir_mult': vir_mult}})
            

In [5]:
# check how many scenarios there are
len(scenario_model_args)

18

In [6]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/09/19 11:32:23 INFO|============================================================
2022/09/19 11:32:23 INFO|============================================================
2022/09/19 11:32:23 INFO|============================================================
2022/09/19 11:32:23 INFO|============================================================
2022/09/19 11:32:23 INFO|============================================================
2022/09/19 11:32:23 INFO|============================================================
2022/09/19 11:32:26 INFO|-----------|{'beta_mult': 0.95, 'booster_mult': 0, 'vir_mult': 2.38} Retrieving vaccinations data
2022/09/19 11:32:26 INFO|-----------|{'beta_mult': 0.95, 'booster_mult': 0, 'vir_mult': 1} Retrieving vaccinations data
2022/09/19 11:32:26 INFO|-----------|{'beta_mult': 0.95, 'booster_mult': 0.5, 'vir_mult': 2.38} Retrieving vaccinations data
2022/09/19 11:32:26 INFO|-----------|{'beta_mult': 0.95, 'booster_mult': 1, 'vir_mult': 1} Retrieving vaccinations d

2022/09/19 11:34:36 INFO|--------|{'beta_mult': 0.95, 'booster_mult': 1, 'vir_mult': 1, 'fit_batch': '0'}: Transmission control fit 1/5 completed in 19.214053334000013 seconds: {767: {'co': 0.8562260979797671}, 781: {'co': 0.8605403634198211}, 795: {'co': 0.8404136448687262}, 809: {'co': 0.834800610162307}, 823: {'co': 0.8363841618839408}, 837: {'co': 0.8244293823686178}}
2022/09/19 11:34:36 INFO|---------|{'beta_mult': 0.95, 'booster_mult': 1, 'vir_mult': 1, 'fit_batch': '0'}: Running forward sim
2022/09/19 11:34:36 INFO|--------|{'beta_mult': 0.95, 'booster_mult': 0.5, 'vir_mult': 1, 'fit_batch': '0'}: Transmission control fit 1/5 completed in 19.15884525 seconds: {767: {'co': 0.8562260979797671}, 781: {'co': 0.8605403634198211}, 795: {'co': 0.8404136448687262}, 809: {'co': 0.834800610162307}, 823: {'co': 0.8363841618839408}, 837: {'co': 0.8244293823686178}}
2022/09/19 11:34:36 INFO|---------|{'beta_mult': 0.95, 'booster_mult': 0.5, 'vir_mult': 1, 'fit_batch': '0'}: Running forward s

2022/09/19 11:35:24 INFO|--------|{'beta_mult': 0.95, 'booster_mult': 0, 'vir_mult': 2.38, 'fit_batch': '3'}: Transmission control fit 4/5 completed in 14.01155566700001 seconds: {851: {'co': 0.8146364809921849}, 865: {'co': 0.8427280981175708}, 879: {'co': 0.8314757024127674}, 893: {'co': 0.8296424306082901}, 907: {'co': 0.8377956203151486}, 921: {'co': 0.8323610118418714}}
2022/09/19 11:35:24 INFO|---------|{'beta_mult': 0.95, 'booster_mult': 0, 'vir_mult': 2.38, 'fit_batch': '3'}: Running forward sim
2022/09/19 11:35:24 INFO|--------|{'beta_mult': 0.95, 'booster_mult': 1, 'vir_mult': 1, 'fit_batch': '3'}: Transmission control fit 4/5 completed in 14.112741375000013 seconds: {851: {'co': 0.8146364809921849}, 865: {'co': 0.8427280981175708}, 879: {'co': 0.8314757024127674}, 893: {'co': 0.8296424306082901}, 907: {'co': 0.8377956203151486}, 921: {'co': 0.8323610118418714}}
2022/09/19 11:35:24 INFO|---------|{'beta_mult': 0.95, 'booster_mult': 1, 'vir_mult': 1, 'fit_batch': '3'}: Running

2022/09/19 11:35:43 INFO|--------|{'beta_mult': 0.95, 'booster_mult': 0.5, 'vir_mult': 1, 'fit_batch': '4'}: Transmission control fit 5/5 completed in 16.70118174999999 seconds: {865: {'co': 0.8422800686511566}, 879: {'co': 0.8316343343443077}, 893: {'co': 0.8298290972305365}, 907: {'co': 0.8370725151068286}, 921: {'co': 0.8369348052916197}, 935: {'co': 0.8241006490732329}}
2022/09/19 11:35:43 INFO|---------|{'beta_mult': 0.95, 'booster_mult': 0.5, 'vir_mult': 1, 'fit_batch': '4'}: Running forward sim
2022/09/19 11:35:44 INFO|--------|{'beta_mult': 0.95, 'booster_mult': 0.5, 'vir_mult': 2.38, 'fit_batch': '4'}: Transmission control fit 5/5 completed in 16.867335500000024 seconds: {865: {'co': 0.8422800686511566}, 879: {'co': 0.8316343343443077}, 893: {'co': 0.8298290972305365}, 907: {'co': 0.8370725151068286}, 921: {'co': 0.8369348052916197}, 935: {'co': 0.8241006490732329}}
2022/09/19 11:35:44 INFO|---------|{'beta_mult': 0.95, 'booster_mult': 0.5, 'vir_mult': 2.38, 'fit_batch': '4'}:

2022/09/19 11:35:47 INFO|============================================================
2022/09/19 11:35:48 INFO|-----------|{'beta_mult': 1.0, 'booster_mult': 0, 'vir_mult': 1} Retrieving vaccinations data
2022/09/19 11:35:48 INFO|============================================================
2022/09/19 11:35:49 INFO|-----------|{'beta_mult': 1.0, 'booster_mult': 0, 'vir_mult': 2.38} Retrieving vaccinations data
2022/09/19 11:35:50 INFO|============================================================
2022/09/19 11:35:51 INFO|-----------|{'beta_mult': 1.0, 'booster_mult': 0.5, 'vir_mult': 1} Retrieving vaccinations data
2022/09/19 11:35:51 INFO|-----------|{'beta_mult': 1.0, 'booster_mult': 0, 'vir_mult': 2.38} Constructing vaccination projections
2022/09/19 11:35:51 INFO|============================================================
2022/09/19 11:35:52 INFO|-----------|{'beta_mult': 1.0, 'booster_mult': 0.5, 'vir_mult': 2.38} Retrieving vaccinations data
2022/09/19 11:35:52 INFO|-----------|{'b

...

2022/09/19 11:39:32 INFO|-----------|{'beta_mult': 1.05, 'booster_mult': 0, 'vir_mult': 1} Constructing vaccination projections
2022/09/19 11:39:33 INFO|============================================================
2022/09/19 11:39:33 INFO|--------|{'beta_mult': 1.0, 'booster_mult': 1, 'vir_mult': 2.38, 'fit_batch': '4', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8306366865716341}, 14: {'co': 0.698195952820626}, 28: {'co': 0.5650437905410518}, 42: {'co': 1.2363018891185438e-15}, 56: {'co': 0.6966529100887137}, 70: {'co': 0.8332726903091638}, 84: {'co': 0.8296248539147445}, 98: {'co': 0.8316924950725628}, 112: {'co': 0.8774896861272652}, 126: {'co': 0.8243044312292909}, 140: {'co': 0.7604527585594816}, 154: {'co': 0.6712880198002112}, 168: {'co': 0.8430551350662076}, 182: {'co': 0.7826363491018719}, 196: {'co': 0.8191511062080898}, 210: {'co': 0.7726071132590828}, 224: {'co': 0.7713215726245787}, 238: {'co': 0.7185439635321001}, 252: {'co': 0.7220971480014075}, 266: {'co': 0.68507768404

2022/09/19 11:41:31 INFO|--------|{'beta_mult': 1.05, 'booster_mult': 0.5, 'vir_mult': 2.38} Model solved in 4.866704667000022 seconds.
2022/09/19 11:41:31 INFO|--------|{'beta_mult': 1.05, 'booster_mult': 0.5, 'vir_mult': 2.38} Will fit 5 times
2022/09/19 11:41:33 INFO|--------|{'beta_mult': 1.05, 'booster_mult': 1, 'vir_mult': 1} Model solved in 4.915572417000021 seconds.
2022/09/19 11:41:33 INFO|--------|{'beta_mult': 1.05, 'booster_mult': 1, 'vir_mult': 1} Will fit 5 times
2022/09/19 11:41:35 INFO|--------|{'beta_mult': 1.05, 'booster_mult': 1, 'vir_mult': 2.38} Model solved in 4.727501332999964 seconds.
2022/09/19 11:41:35 INFO|--------|{'beta_mult': 1.05, 'booster_mult': 1, 'vir_mult': 2.38} Will fit 5 times
2022/09/19 11:41:44 INFO|--------|{'beta_mult': 1.05, 'booster_mult': 0, 'vir_mult': 1, 'fit_batch': '0'}: Transmission control fit 1/5 completed in 18.486798333000024 seconds: {767: {'co': 0.8562278508174602}, 781: {'co': 0.8605343861012077}, 795: {'co': 0.8404247174839354},

2022/09/19 11:42:26 INFO|--------|{'beta_mult': 1.05, 'booster_mult': 0.5, 'vir_mult': 2.38, 'fit_batch': '2'}: Transmission control fit 3/5 completed in 15.543056750000005 seconds: {823: {'co': 0.8345286929583162}, 837: {'co': 0.8340031460887667}, 851: {'co': 0.8247477185629741}, 865: {'co': 0.8607617622180253}, 879: {'co': 0.8551466438260575}, 893: {'co': 0.8587741135035005}}
2022/09/19 11:42:26 INFO|---------|{'beta_mult': 1.05, 'booster_mult': 0.5, 'vir_mult': 2.38, 'fit_batch': '2'}: Running forward sim
2022/09/19 11:42:28 INFO|--------|{'beta_mult': 1.05, 'booster_mult': 1, 'vir_mult': 1, 'fit_batch': '2'}: Transmission control fit 3/5 completed in 15.562098792000029 seconds: {823: {'co': 0.8345286929583162}, 837: {'co': 0.8340031460887667}, 851: {'co': 0.8247477185629741}, 865: {'co': 0.8607617622180253}, 879: {'co': 0.8551466438260575}, 893: {'co': 0.8587741135035005}}
2022/09/19 11:42:28 INFO|---------|{'beta_mult': 1.05, 'booster_mult': 1, 'vir_mult': 1, 'fit_batch': '2'}: Ru

2022/09/19 11:43:00 INFO|--------|{'beta_mult': 1.05, 'booster_mult': 0.5, 'vir_mult': 1, 'fit_batch': '4'}: Transmission control fit 5/5 completed in 16.08633574999999 seconds: {865: {'co': 0.8597702427761388}, 879: {'co': 0.8564365022722875}, 893: {'co': 0.8555957481622144}, 907: {'co': 0.8593206218856286}, 921: {'co': 0.8570186303305475}, 935: {'co': 0.8417474038224001}}
2022/09/19 11:43:00 INFO|---------|{'beta_mult': 1.05, 'booster_mult': 0.5, 'vir_mult': 1, 'fit_batch': '4'}: Running forward sim
2022/09/19 11:43:02 INFO|--------|{'beta_mult': 1.05, 'booster_mult': 0.5, 'vir_mult': 2.38, 'fit_batch': '4'}: Transmission control fit 5/5 completed in 16.353060916000004 seconds: {865: {'co': 0.8597702427761388}, 879: {'co': 0.8564365022722875}, 893: {'co': 0.8555957481622144}, 907: {'co': 0.8593206218856286}, 921: {'co': 0.8570186303305475}, 935: {'co': 0.8417474038224001}}
2022/09/19 11:43:02 INFO|---------|{'beta_mult': 1.05, 'booster_mult': 0.5, 'vir_mult': 2.38, 'fit_batch': '4'}:

2022/09/19 11:43:06 INFO|--------|{'beta_mult': 1.05, 'booster_mult': 1, 'vir_mult': 2.38, 'fit_batch': '4', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8306366865716341}, 14: {'co': 0.698195952820626}, 28: {'co': 0.5650437905410518}, 42: {'co': 1.2363018891185438e-15}, 56: {'co': 0.6966529100887137}, 70: {'co': 0.8332726903091638}, 84: {'co': 0.8296248539147445}, 98: {'co': 0.8316924950725628}, 112: {'co': 0.8774896861272652}, 126: {'co': 0.8243044312292909}, 140: {'co': 0.7604527585594816}, 154: {'co': 0.6712880198002112}, 168: {'co': 0.8430551350662076}, 182: {'co': 0.7826363491018719}, 196: {'co': 0.8191511062080898}, 210: {'co': 0.7726071132590828}, 224: {'co': 0.7713215726245787}, 238: {'co': 0.7185439635321001}, 252: {'co': 0.7220971480014075}, 266: {'co': 0.6850776840450525}, 280: {'co': 0.7198378028999177}, 294: {'co': 0.7589492742542242}, 308: {'co': 0.8220448705663365}, 322: {'co': 0.8119733884357985}, 336: {'co': 0.7821365886735826}, 350: {'co': 0.7958739060541818}, 364: {'

### Run the report for each fit model

In [7]:
# here you can also specify which variants you want to calculate immunity for
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir,
                           prep_model=False, solve_model=True, immun_variants=['ba45', 'vx'], from_date='2022-01-01')

2022/09/19 11:48:36 INFO|============================================================
2022/09/19 11:48:36 INFO|--------|Solving model
2022/09/19 11:48:37 INFO|============================================================
2022/09/19 11:48:37 INFO|--------|Solving model
2022/09/19 11:48:38 INFO|============================================================
2022/09/19 11:48:38 INFO|--------|Solving model
2022/09/19 11:48:39 INFO|============================================================
2022/09/19 11:48:39 INFO|--------|Solving model
2022/09/19 11:48:40 INFO|============================================================
2022/09/19 11:48:40 INFO|--------|Solving model
2022/09/19 11:48:41 INFO|============================================================
2022/09/19 11:48:41 INFO|--------|Solving model
2022/09/19 11:48:41 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/09/19 

In [8]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/09/19 12:00:40 INFO|Projecting
2022/09/19 12:00:40 INFO|
2022/09/19 12:01:04 INFO|{'beta_mult': 0.95, 'booster_mult': 0, 'vir_mult': 1, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/19 12:01:06 INFO|
2022/09/19 12:01:32 INFO|{'beta_mult': 0.95, 'booster_mult': 0, 'vir_mult': 2.38, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/19 12:01:34 INFO|
2022/09/19 12:01:58 INFO|{'beta_mult': 0.95, 'booster_mult': 0.5, 'vir_mult': 1, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/19 12:02:00 INFO|
2022/09/19 12:02:25 INFO|{'beta_mult': 0.95, 'booster_mult': 0.5, 'vir_mult': 2.38, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/19 12:02:27 INFO|
2022/09/19 12:02:50 INFO|{'beta_mult': 0.95, 'booster_mult': 1, 'vir_mult': 1, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/19 12:02:52 INFO|
2022/09/19 12:03:16 INFO|{'beta_mult': 0.95, 'booster_mult': 1, 'vir_mult': 2.38, 'fit_batch': '4', 'run_type': '